In [1]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
from mpl_toolkits.axes_grid1 import make_axes_locatable
import tensorflow as tf
import os

2023-05-15 16:18:31.896122: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-15 16:18:31.899761: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-15 16:18:31.899775: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Open the netCDF file
nc_file = nc.Dataset('fordle/ela_data/train/ALP00_A78_C0/ex.nc', 'r')
# Get the number of records in the file
num_records = nc_file.dimensions['idx'].size

In [3]:
# Define the index to split the file
split_index = num_records*0.8  # Change this to your desired split index

In [17]:
# Create two separate netCDF files
part1_file = nc.Dataset('part1.nc', 'w', format='NETCDF4')
part2_file = nc.Dataset('part2.nc', 'w', format='NETCDF4')


In [18]:
# Copy global attributes from the original file
part1_file.setncatts(nc_file.__dict__)
part2_file.setncatts(nc_file.__dict__)

In [19]:
# Create dimensions in the new files 
exlude=['x']
for name, dimension in nc_file.dimensions.items():
    exlude.append(dimension)
    if name == 'idx':
        part1_file.createDimension(name, split_index)
        part2_file.createDimension(name, num_records - split_index)
    else:
        part1_file.createDimension(name, len(dimension))
        part2_file.createDimension(name, len(dimension))


In [7]:
# Copy variables from the original file to the new files
for name, variable in nc_file.variables.items():
    # Create variables in the new files
    var1 = part1_file.createVariable(name, variable.dtype, variable.dimensions)
    var2 = part2_file.createVariable(name, variable.dtype, variable.dimensions)

    # Copy variable attributes
    var1.setncatts(variable.__dict__)
    var2.setncatts(variable.__dict__)
    print(name)
    
    # Copy variable data based on the split index
    var1[name][:] = nc_file[name][:split_index]
    var2[name][:] = nc_file[name][split_index:]


x


TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
# Close the files
nc_file.close()
part1_file.close()
part2_file.close()

In [8]:
# Copy variables from the original file to the new files
for name, variable in nc_file.variables.items():
     print(name)

x
y
idx
max_thk
topg
ela


In [13]:
part2_file

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): x(451), y(301), idx(200)
    variables(dimensions): 
    groups: 

In [14]:
var1

<class 'netCDF4._netCDF4.Variable'>
float32 x(x)
    _FillValue: nan
unlimited dimensions: 
current shape = (451,)
filling on

In [22]:
len(dimension)

1000

In [10]:
nc_file.variables.items()

dict_items([('x', <class 'netCDF4._netCDF4.Variable'>
float32 x(x)
    _FillValue: nan
unlimited dimensions: 
current shape = (451,)
filling on), ('y', <class 'netCDF4._netCDF4.Variable'>
float32 y(y)
    _FillValue: nan
unlimited dimensions: 
current shape = (301,)
filling on), ('idx', <class 'netCDF4._netCDF4.Variable'>
float64 idx(idx)
    _FillValue: nan
unlimited dimensions: 
current shape = (1000,)
filling on), ('max_thk', <class 'netCDF4._netCDF4.Variable'>
float32 max_thk(idx, y, x)
    _FillValue: nan
unlimited dimensions: 
current shape = (1000, 301, 451)
filling on), ('topg', <class 'netCDF4._netCDF4.Variable'>
float32 topg(idx, y, x)
    _FillValue: nan
unlimited dimensions: 
current shape = (1000, 301, 451)
filling on), ('ela', <class 'netCDF4._netCDF4.Variable'>
float32 ela(idx, y, x)
    _FillValue: nan
unlimited dimensions: 
current shape = (1000, 301, 451)
filling on)])